In [1]:
!pip install kaggle

In [7]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [8]:
import os
import shutil

# Create the .kaggle directory if it doesn't exist
os.makedirs("/root/.kaggle", exist_ok=True)

# Move the uploaded kaggle.json to the correct place
shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")

# Set strict permissions (required)
os.chmod("/root/.kaggle/kaggle.json", 600)

In [11]:
!kaggle datasets download -d ericanacletoribeiro/cicids2017-cleaned-and-preprocessed
!unzip cicids2017-cleaned-and-preprocessed.zip

Dataset URL: https://www.kaggle.com/datasets/ericanacletoribeiro/cicids2017-cleaned-and-preprocessed
License(s): CC0-1.0
 59% 118M/200M [00:00<00:00, 1.23GB/s]
100% 200M/200M [00:00<00:00, 704MB/s] 
Archive:  cicids2017-cleaned-and-preprocessed.zip
  inflating: cicids2017_cleaned.csv  


In [12]:
import pandas as pd

df = pd.read_csv("cicids2017_cleaned.csv")

df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,...,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Max,Active Min,Idle Mean,Idle Max,Idle Min,Attack Type
0,22,1266342,41,2664,456,0,64.975610,109.864573,976,0,...,243,24,32,0.0,0,0,0.0,0,0,Normal Traffic
1,22,1319353,41,2664,456,0,64.975610,109.864573,976,0,...,243,24,32,0.0,0,0,0.0,0,0,Normal Traffic
2,22,160,1,0,0,0,0.000000,0.000000,0,0,...,243,0,32,0.0,0,0,0.0,0,0,Normal Traffic
3,22,1303488,41,2728,456,0,66.536585,110.129945,976,0,...,243,24,32,0.0,0,0,0.0,0,0,Normal Traffic
4,35396,77,1,0,0,0,0.000000,0.000000,0,0,...,290,0,32,0.0,0,0,0.0,0,0,Normal Traffic


In [13]:
print("Rows and Columns:",df.shape)

Rows and Columns: (2520751, 53)


In [14]:
print("Column Names:\n",df.columns)

Column Names:
 Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Length of Fwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd Header Length', 'Bwd Header Length',
       'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length',
       'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
       'Packet Length Variance', 'FIN Flag Count', 'PSH Flag Count',
       'ACK Flag Count', 'Average Packet Size', 'Subflow Fwd Bytes',
       'Init_Win_bytes_forward', 'Init_Win_bytes_backwar

In [15]:
df.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,...,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Max,Active Min,Idle Mean,Idle Max,Idle Min
count,2.520751e+06,2.520751e+06,2.520751e+06,2.520751e+06,2.520751e+06,2.520751e+06,2.520751e+06,2.520751e+06,2.520751e+06,2.520751e+06,...,2.520751e+06,2.520751e+06,2.520751e+06,2.520751e+06,2.520751e+06,2.520751e+06,2.520751e+06,2.520751e+06,2.520751e+06,2.520751e+06
mean,8.690744e+03,1.659032e+07,1.025880e+01,6.065243e+02,2.311993e+02,1.920377e+01,6.350189e+01,7.732029e+01,9.749253e+02,4.316019e+01,...,7.265707e+03,2.228741e+03,5.997266e+00,-3.082291e+03,9.152230e+04,1.719410e+05,6.542438e+04,9.337159e+06,9.763454e+06,8.892545e+06
std,1.901294e+04,3.523140e+07,7.943824e+02,1.011595e+04,7.563122e+02,6.079887e+01,1.955512e+02,2.968824e+02,2.038152e+03,7.088017e+01,...,1.459805e+04,8.924482e+03,6.743909e+02,1.149769e+06,6.858746e+05,1.084664e+06,6.106447e+05,2.484810e+07,2.561725e+07,2.458139e+07
min,0.000000e+00,-1.300000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-1.000000e+00,-1.000000e+00,0.000000e+00,-5.368707e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.300000e+01,2.080000e+02,2.000000e+00,1.200000e+01,6.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,...,-1.000000e+00,-1.000000e+00,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,8.000000e+01,5.062000e+04,2.000000e+00,6.600000e+01,4.000000e+01,2.000000e+00,3.625000e+01,0.000000e+00,9.700000e+01,0.000000e+00,...,2.510000e+02,0.000000e+00,1.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.430000e+02,5.332968e+06,6.000000e+00,3.320000e+02,2.020000e+02,3.700000e+01,5.200000e+01,7.419280e+01,7.460000e+02,8.200000e+01,...,8.192000e+03,2.350000e+02,3.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553500e+04,1.200000e+08,2.197590e+05,1.290000e+07,2.482000e+04,2.325000e+03,5.940857e+03,7.125597e+03,1.953000e+04,2.896000e+03,...,6.553500e+04,6.553500e+04,2.135570e+05,1.380000e+02,1.100000e+08,1.100000e+08,1.100000e+08,1.200000e+08,1.200000e+08,1.200000e+08


In [16]:
import numpy as np

df_cleaned = df.copy()

numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns

df_cleaned[numeric_cols] = df_cleaned[numeric_cols].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
print("Any negatives left?\n", (df_cleaned[numeric_cols] < 0).sum().sum())

Any negatives left?
 0


In [17]:
missing_values = df_cleaned.isnull().sum()
print("Missing Values:\n", missing_values)

missing_values = missing_values[missing_values > 0]
print("\nColumns with Missing Values:\n", missing_values)

Missing Values:
 Destination Port               0
Flow Duration                  0
Total Fwd Packets              0
Total Length of Fwd Packets    0
Fwd Packet Length Max          0
Fwd Packet Length Min          0
Fwd Packet Length Mean         0
Fwd Packet Length Std          0
Bwd Packet Length Max          0
Bwd Packet Length Min          0
Bwd Packet Length Mean         0
Bwd Packet Length Std          0
Flow Bytes/s                   0
Flow Packets/s                 0
Flow IAT Mean                  0
Flow IAT Std                   0
Flow IAT Max                   0
Flow IAT Min                   0
Fwd IAT Total                  0
Fwd IAT Mean                   0
Fwd IAT Std                    0
Fwd IAT Max                    0
Fwd IAT Min                    0
Bwd IAT Total                  0
Bwd IAT Mean                   0
Bwd IAT Std                    0
Bwd IAT Max                    0
Bwd IAT Min                    0
Fwd Header Length              0
Bwd Header Length         

In [18]:
from sklearn.preprocessing import LabelEncoder

#create a labelencoder object to convert strings to numbers
encoder = LabelEncoder()

#encode the attack type column
df_cleaned['Attack type'] = encoder.fit_transform(df_cleaned['Attack Type'])

#see what the encoder did
print("Label mapping:", dict(zip(encoder.classes_, encoder.transform(encoder.classes_))))

Label mapping: {'Bots': np.int64(0), 'Brute Force': np.int64(1), 'DDoS': np.int64(2), 'DoS': np.int64(3), 'Normal Traffic': np.int64(4), 'Port Scanning': np.int64(5), 'Web Attacks': np.int64(6)}


In [19]:
from sklearn.model_selection import train_test_split

#split the data into training and testing sets

x = df_cleaned.drop(['Attack Type', 'Attack type'], axis=1)
y = df_cleaned['Attack type']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)# This will print the number of rows and columns in the training and testing sets

print("Training Features (X_train):", X_train.shape)  # (rows, columns)
print("Testing Features (X_test):", X_test.shape)

print("Training Labels (y_train):", y_train.shape)
print("Testing Labels (y_test):", y_test.shape)

Training Features (X_train): (2016600, 52)
Testing Features (X_test): (504151, 52)
Training Labels (y_train): (2016600,)
Testing Labels (y_test): (504151,)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

# Load the cleaned dataset
df = pd.read_csv('/content/drive/MyDrive/df_cleaned.csv')

# Check the first few rows
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Max,Active Min,Idle Mean,Idle Max,Idle Min,Attack Type,Attack type
0,0.000336,1.055296e-02,0.000182,0.000207,0.018372,0.0,0.010937,0.015418,0.049974,0.0,...,0.000112,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Normal Traffic,4
1,0.000336,1.099472e-02,0.000182,0.000207,0.018372,0.0,0.010937,0.015418,0.049974,0.0,...,0.000112,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Normal Traffic,4
2,0.000336,1.441667e-06,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Normal Traffic,4
3,0.000336,1.086251e-02,0.000182,0.000211,0.018372,0.0,0.011200,0.015456,0.049974,0.0,...,0.000112,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Normal Traffic,4
4,0.540108,7.499999e-07,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Normal Traffic,4


In [3]:
#  Sample 2000 rows from each available attack type
df_sampled = df.groupby('Attack Type').apply(lambda x: x.sample(n=2000, random_state=42)).reset_index(drop=True)

#  Check shape (should be 5 classes × 2000 = 10,000 rows)
print("Sampled data shape:", df_sampled.shape)

Sampled data shape: (10000, 54)


/tmp/ipython-input-3-4128910032.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df.groupby('Attack Type').apply(lambda x: x.sample(n=2000, random_state=42)).reset_index(drop=True)


In [4]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder
label_encoder = LabelEncoder()

# Transform "Attack Type" text labels into numeric values
df_sampled['Label'] = label_encoder.fit_transform(df_sampled['Attack Type'])

# Let's see the mapping
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label mapping:", label_mapping)

Label mapping: {'Brute Force': np.int64(0), 'DDoS': np.int64(1), 'Normal Traffic': np.int64(2), 'Port Scanning': np.int64(3), 'Web Attacks': np.int64(4)}


In [5]:
#  Drop the original text label (we already converted it)
X = df_sampled.drop(['Attack Type', 'Label'], axis=1)
y = df_sampled['Label']

In [6]:
from sklearn.model_selection import train_test_split

# 80% for training, 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)

Training set size: (8000, 53)
Test set size: (2000, 53)


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# to Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

#  Train the model
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Show evaluation results
print("Classification Report:\n", classification_report(y_test, predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, predictions))

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       410
           1       1.00      1.00      1.00       393
           2       1.00      1.00      1.00       395
           3       1.00      1.00      1.00       415
           4       1.00      1.00      1.00       387

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000

Confusion Matrix:
 [[410   0   0   0   0]
 [  0 393   0   0   0]
 [  0   0 395   0   0]
 [  0   0   0 415   0]
 [  0   0   0   0 387]]


In [8]:
import joblib

# Save the model
joblib.dump(model, '/content/drive/MyDrive/random_forest_nids_model.pkl')

['/content/drive/MyDrive/random_forest_nids_model.pkl']

In [9]:
# Sample prediction
sample = X_test.iloc[0].values.reshape(1, -1)
predicted_class = model.predict(sample)[0]

# Decode label
label_map = {0: 'Bots', 1: 'Brute Force', 2: 'DDoS', 3: 'DoS', 4: 'Normal Traffic', 5: 'Port Scanning', 6: 'Web Attacks'}
print("Predicted:", label_map[predicted_class])

Predicted: DoS


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [10]:
sample = X_test.iloc[[0]]  # Double brackets keep it as a DataFrame
predicted_class = model.predict(sample)[0]

label_map = {0: 'Bots', 1: 'Brute Force', 2: 'DDoS', 3: 'DoS', 4: 'Normal Traffic', 5: 'Port Scanning', 6: 'Web Attacks'}
print("Predicted:", label_map[predicted_class])

Predicted: DoS


In [11]:
import joblib

# Save the model as a file
joblib.dump(model, '/content/drive/MyDrive/nids_model.pkl')

['/content/drive/MyDrive/nids_model.pkl']

In [12]:
joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']